In [ ]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score




In [ ]:
population = pd.read_csv('population.csv')
#filter out non-country data
population = population.loc[0:216]
population = population.drop(index=213)
population.reset_index(drop=True,inplace=True)
#making the dataset, x(30 years) and y(next 10 years)
first_thirty = []
last_fifteen = []
last_thirty = []

for i in population.index:
  first_list = list(population.iloc[i,12:47].astype(float))
  second_list = list(population.iloc[i,22:57].astype(float))
  third_list = list(population.iloc[i,32:67].astype(float))
  for j in range(16,46,3):
    first_list.append(float(population.iloc[i,j+3]) - float(population.iloc[i,j]))
  for j in range(26,56,3):
    second_list.append(float(population.iloc[i,j+3]) - float(population.iloc[i,j]))
  for j in range(36,66,3):
    third_list.append(float(population.iloc[i,j+3]) - float(population.iloc[i,j]))
  for j in range(16,46,5):
    first_list.append(float(population.iloc[i,j+5]) - float(population.iloc[i,j]))
    first_list.append((float(population.iloc[i,j+5]) - float(population.iloc[i,j])) / float(population.iloc[i,j]))
  for j in range(26,56,5):
    second_list.append(float(population.iloc[i,j+5]) - float(population.iloc[i,j]))
    second_list.append((float(population.iloc[i,j+5]) - float(population.iloc[i,j])) / float(population.iloc[i,j]))
  for j in range(36,66,5):
    third_list.append(float(population.iloc[i,j+5]) - float(population.iloc[i,j]))
    third_list.append((float(population.iloc[i,j+5]) - float(population.iloc[i,j])) / float(population.iloc[i,j]))
  last_five_years_first = first_list[-2]/5
  last_five_years_second = second_list[-2]/5
  last_five_years_third = third_list[-2]/5
  last_ten_years_first = (first_list[-2]+first_list[-4])/10
  last_ten_years_second = (second_list[-2]+second_list[-4])/10
  last_ten_years_third = (third_list[-2]+third_list[-4])/10
  for j in range(1,11):
    if j <= 5:
      first_list.append((last_five_years_first * j) + first_list[34])
      second_list.append((last_five_years_second * j) + second_list[34])
      third_list.append((last_five_years_third * j) + third_list[34])
    else:
      first_list.append((last_ten_years_first * j) + first_list[34])
      second_list.append((last_ten_years_second * j) + second_list[34])
      third_list.append((last_ten_years_third * j) + third_list[34])


  first_thirty.append(first_list)
  first_thirty.append(second_list)
  last_fifteen.append(list(population.iloc[i,47:57].astype(float)))
  last_fifteen.append(list(population.iloc[i,57:67].astype(float)))
  last_thirty.append(third_list)




In [ ]:
first_thirty[422]

[40145287.0,
 41015865.0,
 41928849.0,
 42916624.0,
 43906019.0,
 44891281.0,
 45898698.0,
 46969616.0,
 48163573.0,
 49418150.0,
 50701458.0,
 51831389.0,
 52968270.0,
 54280394.0,
 55632153.0,
 57011444.0,
 58406863.0,
 59811313.0,
 61221107.0,
 62630787.0,
 64037514.0,
 65466361.0,
 66912613.0,
 68358820.0,
 69788747.0,
 71176405.0,
 72501087.0,
 73759110.0,
 74946448.0,
 76058603.0,
 77128424.0,
 78123713.0,
 79001142.0,
 79817777.0,
 80642308.0,
 3063597.0,
 3731842.0,
 3578936.0,
 4126469.0,
 4223924.0,
 4281826.0,
 4263792.0,
 3770043.0,
 3177265.0,
 2518595.0,
 5512131.0,
 0.12554385766561985,
 6214003.0,
 0.12574333519162495,
 6998634.0,
 0.12580196204162725,
 7157960.0,
 0.11428820142400574,
 6269856.0,
 0.08984050107677101,
 4583705.0,
 0.060265437691512685,
 81559049.0,
 82475790.0,
 83392531.0,
 84309272.0,
 85226013.0,
 87154444.6,
 88239800.7,
 89325156.8,
 90410512.9,
 91495869.0]

In [ ]:
#creating and fitting the model
x_train, x_test, y_train, y_test = model_selection.train_test_split(first_thirty, last_fifteen, test_size = 0.15, random_state=1)

population_predictor = MLPRegressor( max_iter = 5000)
population_predictor.fit(x_train, y_train)

MLPRegressor(max_iter=5000)

In [ ]:
#validation
y_pred = population_predictor.predict(x_test)
score = population_predictor.score(x_test, y_test)
r2_score_population = r2_score(y_test,y_pred)
print(score, r2_score_population)

0.9998913483410025 0.9998913483410025


In [ ]:
#make population predictions for the next 15 years

next_fifteen = population_predictor.predict(last_thirty)
population_predictions = pd.DataFrame(next_fifteen, columns=range(2023,2033))
population_predictions = population[['Country Name']].join(population_predictions)
population_predictions[population_predictions['Country Name'] == 'Brazil']

,Country Name,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032
26,Brazil,2.176532e+08,2.171807e+08,2.202505e+08,2.203484e+08,2.214378e+08,2.220048e+08,2.234269e+08,2.230967e+08,2.240999e+08,2.252162e+08


In [ ]:
last_thirty[26]

[145253973.0,
 148003411.0,
 150706446.0,
 153336445.0,
 155900790.0,
 158440875.0,
 160980472.0,
 163515328.0,
 166037122.0,
 168546707.0,
 171039804.0,
 173486281.0,
 175873720.0,
 178211881.0,
 180476685.0,
 182629278.0,
 184722043.0,
 186797334.0,
 188820682.0,
 190779453.0,
 192672317.0,
 194517549.0,
 196353492.0,
 198185302.0,
 199977707.0,
 201721767.0,
 203459650.0,
 205188205.0,
 206859578.0,
 208504960.0,
 210166592.0,
 211782878.0,
 213196304.0,
 214326223.0,
 215313498.0,
 7614538.0,
 7524476.0,
 7172077.0,
 6510162.0,
 6057410.0,
 5574039.0,
 5368275.0,
 5137811.0,
 4923300.0,
 3530620.0,
 12645917.0,
 0.08111515663262514,
 11929978.0,
 0.07078143626976942,
 10302768.0,
 0.057086420885888944,
 9198254.0,
 0.048214070516283535,
 8527253.0,
 0.04264101798106926,
 6808538.0,
 0.032654081706257734,
 216675205.6,
 218036913.2,
 219398620.8,
 220760328.4,
 222122036.0,
 224514972.6,
 226048551.7,
 227582130.8,
 229115709.9,
 230649289.0]